# Очистка текстов и создание обучающего датасета

In [ ]:
import pandas as pd
import numpy as np
import json
import re
from nltk.stem.snowball import SnowballStemmer

In [ ]:
issuers = pd.read_excel('issuers.xlsx')
issuers.head(2)

,Unnamed: 0,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker
0,0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN
1,1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN


In [ ]:
syn = pd.read_excel('synonyms.xlsx')
syn.head(2)

,issuerid,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
names_texts = pd.read_pickle('mentions texts.pickle')
names_texts.head(2)

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False


In [ ]:
sent_texts = pd.read_pickle('sentiment_texts.pickle')
sent_texts.head(2)

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False


### Очистка текста

In [ ]:
def clear_text(df):
    remove_question = lambda text: re.sub(r'\?{2,}', '', text)
    df['MessageTextClean'] = df['MessageText'].apply(remove_question)
    remove_u200b = lambda text: re.sub(r'\u200b', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_u200b)
    remove_backslash = lambda text: re.sub(r'\\[^ ]*', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_backslash)
    remove_at = lambda text: re.sub(r'\@\S{2,}', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_at)
    remove_site = lambda text: re.sub(r'http\S{2,}', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_site)
    remove_space = lambda text: re.sub(r'\s{2,}', ' ', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_space)
    remove_quotation = lambda text: re.sub(r'\"{2,}', '"', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_quotation)
    return df

In [ ]:
%%time
names_texts = clear_text(names_texts)
sent_texts = clear_text(sent_texts)

CPU times: total: 938 ms
Wall time: 1.06 s


In [ ]:
sent_texts.to_csv('sent_clean2.csv', index=False)

### Формирование списков синонимов

In [ ]:
names = issuers.merge(syn.drop(['EMITENT_FULL_NAME', 'BGTicker'], axis=1), on='issuerid', how='inner')
names.head(2)

,Unnamed: 0,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker,VeryOddCompany,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN,NaN,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
l_syns = ['EMITENT_FULL_NAME', 'BGTicker', 'OtherTicker', 'BGTicker.1']
for i in range(5, 15):
    l_syns.append('Unnamed: '+str(i))
l_syns

['EMITENT_FULL_NAME',
 'BGTicker',
 'OtherTicker',
 'BGTicker.1',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14']

In [ ]:
%%time
names['l_syns'] = names[l_syns].apply(lambda x: [re.sub(r'\s{2,}', '', a) for a in x if str(a) != 'nan'], axis=1)
names['l_syns'] = names['l_syns'].apply(lambda x: list(set(x)))
names['l_syns']

CPU times: total: 0 ns
Wall time: 4.99 ms


0      [Держава, DERZ, DERZHAVA, "Акционерный коммерч...
1      [мкб, CBOM RX, Московский кредитный банк, Cred...
2      [roads Bank, "Российский акционерный коммерчес...
3      [алроса, alrosa, ALRS RX, Акционерная компания...
4      [Акционерный Коммерческий банк "АВАНГАРД" - пу...
                             ...                        
250                         [Хендерсон, Henderson, HNFG]
251                       [Sovcombank, SVCB, Совкомбанк]
252    [АЗС «Трасса», АЗС Трасса, Евротранс, EUTR, АЗ...
253    ["Каршеринг Руссия", ПАО, ПАО "Каршеринг Русси...
254                             [Диасофт, DIAS, Diasoft]
Name: l_syns, Length: 255, dtype: object

In [ ]:
company_names = names[['issuerid', 'l_syns']].copy()
company_names.head()

,issuerid,l_syns
0,1,"[Держава, DERZ, DERZHAVA, ""Акционерный коммерч..."
1,2,"[мкб, CBOM RX, Московский кредитный банк, Cred..."
2,3,"[roads Bank, ""Российский акционерный коммерчес..."
3,4,"[алроса, alrosa, ALRS RX, Акционерная компания..."
4,5,"[Акционерный Коммерческий банк ""АВАНГАРД"" - пу..."


In [ ]:
def ff(x):
    for i in x:
        if ' RX' in i:
            x.append(i.split(' RX')[0])
            break
    return x

In [ ]:
company_names['l_syns'] = company_names['l_syns'].apply(lambda x: ff(x))

In [ ]:
company_names['l_syns'].loc[3]

['алроса',
 'alrosa',
 'ALRS RX',
 'Акционерная компания "АЛРОСА" (публичное акционерное общество)',
 'ALRS']

In [ ]:
company_names.to_pickle('company_names2.pickle')

### Создание обучающего датасета

In [ ]:
company_names = pd.read_pickle('company_names2.pickle')
company_names['l_syns'] = company_names['l_syns'].apply(lambda x: [a.lower() for a in x])
company_names['l_syns'].loc[1]

['cbom rx',
 '"московский кредитный банк" (публичное акционерное общество)',
 'credit bank of moscow',
 'московский кредитный банк',
 'мкб',
 'credit bank',
 'cbom']

In [ ]:
df = names_texts[['issuerid', 'MessageTextClean']].copy()

In [ ]:
df.duplicated().sum()

115

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
stemmer = SnowballStemmer("russian")
company_names_uniq = company_names['issuerid'].unique()
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя0123456789abcdefghijklmnopqrstuvwxyz')

def find_name(text, name, size):
    name = name.lower()
    l = [(m.start(), m.start()+size) for m in re.finditer(name, text)]
    newl = []
    #Проверка, что найденное название не является частью другого слова
    for el in l:
        if (el[0]>=0 and el[1]<len(text)) and (text[el[0]-1] not in alphabet and
                                               text[el[1]] not in alphabet):
            newl.append(el)
    return newl

k = 0
def find_company_names(text, issuerid):
    text = text.lower()
    d = {}
    d['text'] = text
    names = company_names.loc[company_names['issuerid']==issuerid, 'l_syns'].iloc[0]
    l = []
    for name in names:
        l.extend(find_name(text, name, len(name)))
        name1 = stemmer.stem(name)
        if not name.__eq__(name1):
            l.extend(find_name(text, name1, len(name)))
    l = list(set(l))
    #Если сущность входит в другую сущность, то оставляем большую сущность
    newl = []
    for i in l:
        f = True
        for j in l:
            if (i[0]>j[0] and i[1]<=j[1]) or (i[0]>=j[0] and i[1]<j[1]):
                f = False
                break
        if f:
            newl.append(i)
    return newl

def find_companies_names(text):
    global k
    subdf = df.loc[df['MessageTextClean']==text]
    l = []
    d = {}
    for issuerid in company_names_uniq:
        ll = find_company_names(text, issuerid)
        l.extend(ll)
    k += 1
    d['text'] = text
    d['name'] = list(set(l))
    return d

text = df.loc[0, 'MessageTextClean']
jj = find_companies_names(text)
jj

{'text': ' Фокус недели #ФН Сегодня ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ФРС США представит решение по процентной ставке #ФРС #Макро Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность 30 апреля Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR #Отчетность СД НЛМК рассмотрит дивиденды за 1 кв. 2020 #NLMK #Дивиденды Apple 

In [ ]:
print(jj['text'])
l = []
for i in jj['name']:
    l.append(jj['text'][i[0]:i[1]])
print('\nCompanies - ', np.array(l))

 Фокус недели #ФН Сегодня ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ФРС США представит решение по процентной ставке #ФРС #Макро Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность 30 апреля Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR #Отчетность СД НЛМК рассмотрит дивиденды за 1 кв. 2020 #NLMK #Дивиденды Apple опубликует

In [ ]:
names_texts = pd.read_pickle('mentions texts.pickle')
names_texts = clear_text(names_texts)
names_texts.loc[names_texts['MessageTextClean']==jj['text']]

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward,MessageTextClean
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False,Фокус недели #ФН Сегодня ММК опубликует финан...


In [ ]:
from tqdm import tqdm
pp = 0
res = []
uniq_texts = df['MessageTextClean'].unique()
for i in tqdm(range(len(uniq_texts))):
    d = find_companies_names(uniq_texts[i])
    if d:
        res.append(d)
    else:
        pp += 1

In [ ]:
with open('data3_allnames.json', 'w') as f:
    json.dump({'data': res}, f)